In [1]:
import argparse
import logging
import os
import random
import pandas as pd
from dataclasses import dataclass
from itertools import chain
from typing import Optional, Union
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

import datasets
import torch
from datasets import load_dataset, load_metric, DatasetDict
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    PreTrainedTokenizerBase,
    default_data_collator,
    DataCollatorForSeq2Seq,
    AdamW,
    SchedulerType,
    get_scheduler,
    set_seed,
)
from transformers.file_utils import PaddingStrategy
from promptsource.templates import DatasetTemplates
pd.set_option("display.max_rows", 1200)


/opt/conda/envs/tz/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
n = 1000
model_dir = "output_1000_plus"
df = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# accuracy = np.zeros(n-1)
# for i in range(1, n):
#     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
#     accuracy[i-1] = temp['accuracy'].iloc[0]
# model_dir = "output"
# df2 = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# accuracy2 = np.zeros(n-1)
# for i in range(1, n):
#     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
#     accuracy2[i-1] = temp['accuracy'].iloc[0]
# # model_dir = "output2"
# # df3 = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# # accuracy3 = np.zeros(n-1)
# # for i in range(1, n):
# #     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
# #     accuracy3[i-1] = temp['accuracy'].iloc[0]
    

In [ ]:
df.to_csv(f'/home/gikok/{model_dir}/stats.csv')

In [ ]:
n = 342
model_dir = "output_no_GA"
df = pd.read_csv(f'/home/gikok/{model_dir}/stats.csv')
even = np.arange(0, len(df)-1, 2)
even1 = np.arange(1, len(df), 2)
df1 = df.iloc[even]
df2 = df.iloc[even1]

In [ ]:
e = df.groupby(df.index//200).median()
et =  df.groupby(df.index//200).max()
eb =  df.groupby(df.index//200).quantile(0.25)
e2 = df2.groupby(df2.index//200).max()
et2 =  df2.groupby(df2.index//200).quantile(0.75)
eb2 =  df2.groupby(df2.index//200).quantile(0.25)
l = np.arange(len(e))
plt.rcParams['font.size'] = 16
# plt.plot(l, e["training_loss"], label = 'Task 1', c='green')
plt.plot(l, et["test_loss"], c='green', ls='--')
# plt.plot(l, eb["training_loss"], c='green', ls='--')

plt.plot(l, e2["test_loss"], label = 'Task 2', c='red', alpha=0.6)
# plt.plot(l, et2["training_loss"], c='red', ls='--')
# plt.plot(l, eb2["training_loss"], c='red', ls='--')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("2 options")
#plt.ylim(3e-1, 10)
plt.legend()
# plt.yscale('log')
plt.show()
plt.plot(l, abs(e["lm_grad_pct"]), label = 'Task 1')
plt.plot(l, abs(e2["lm_grad_pct"]), label = 'Task 2')
plt.xlabel('Epoch')
plt.yscale('log')
plt.title("Output Vocabulary")
plt.ylabel('<Gradient>')
plt.legend()
#plt.yscale('log')
plt.show()
plt.plot(l, abs(e["em_grad_pct"]), label= 'Task 1')
plt.plot(l, abs(e2["em_grad_pct"]), label='Task 2')
plt.xlabel('Epoch')
plt.ylabel('<Gradient>')
plt.title("Input Vocabulary")
plt.legend()
plt.yscale('log')
plt.show()
plt.plot(l, e["lm_delta_pct"], '.', label = 'LM delta (prompt 1)')
plt.plot(l, e2["lm_delta_pct"], '.', label = 'LM delta (prompt 2)')
plt.xlabel('Epoch')
#plt.ylim([-0.2, 0.2])
plt.ylabel('<Delta> (%)')
plt.legend()
plt.yscale('log')
plt.show()
plt.plot(l, e["em_delta_pct"], '.', label= 'EM delta (prompt 1)')
plt.plot(l, e2["em_delta_pct"], '.', label=' EM delta (prompt 2)')
plt.xlabel('Epoch')
#plt.ylim([-2e-3, 2e-3])
plt.ylabel('<Delta> (%)')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
torch.cuda.device_count()

In [2]:
#Load model and tokenizer

model_name = "/home/gikok/new_model_1mil_data/final_model/"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Model and tokenizer loaded")

model.to('cuda:0')
print("Moved model to GPUs")



Model and tokenizer loaded
Moved model to GPUs


In [71]:
df = pd.read_parquet('data/miniprompts002.parquet.gzip')

In [6]:
emb = list(model.named_parameters())[0][1][-101:-1]

In [4]:
emb.grad

/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  """Entry point for launching an IPython kernel.


In [49]:
print("Test for hierarchy_prompt")
for i in range(10):
    inp = sub_df['input'].iloc[i]
    tar = sub_df['target'].iloc[i]
    inputs = tokenizer.encode(
        inp, return_tensors='pt')
    if len(inputs[0])>150:
        print("INPUT WAS CONCATINATED")
    inputs[0] = inputs[0][:150]
    inputs = inputs.to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(inputs, max_new_tokens=100)
    print(f"********************Example number: {i+1}********************")
    print("Input: " + inp)
    print("Target: " + tar)
    print("Output: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test for hierarchy_prompt
********************Example number: 1********************
Input: item_no 70400448 and item_no 30253067 are similar because they are described by which of the following? hemmes glass, red dish towel, bathroom etagere, room darkening liner, outside lights, kitchen drawer dividers 
Target: 70400448 and 30253067 are similar because they are described by bathroom etagere
Output: kitchen drawer dividers
********************Example number: 2********************
Input: item_no 50313013 and item_no 10495977 are similar because they are described by which of the following? brown bookshelves, ksl lax, nackanas, queens sheets, rugs shag, bench and table
Target: 50313013 and 10495977 are similar because they are described by rugs shag
Output: a bench and table
********************Example number: 3********************
Input: item_no 20252718 and item_no 60252721 are similar because they are described by which of the following? sektion 12" base cabinet, end desk, brimmed kin

In [41]:
items = pd.read_parquet("data/seven_names.parquet.gzip")["item_no"].values.tolist()
np.random.shuffle(items)
#print(items)
inp = f"for item_no: '40415601'. Which of the following is the best match? cabinet describes 40415601, table describes 40415601, bookshelves describes 40415601?"
tar = 'bookshelves describes 40415601'
inp1 = inp  + ', '.join(items)

inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)
print("Input: " + inp)
print("Target: " + tar)
print("Output: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

/opt/conda/envs/tz/lib/python3.7/site-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Input: for item_no: '40415601'. Which of the following is the best match? cabinet describes 40415601, table describes 40415601, bookshelves describes 40415601?
Target: bookshelves describes 40415601
Output: bookshelves describes 40415601


In [65]:
f = 3
sub_df['input'].iloc[f], sub_df['target'].iloc[f]

("description: 'red lantern'\nfor the description above, what item_no is the correct match? 60265634, 09332694, 80290675, 90494950, 19485605, 19456165?",
 '90494950')

In [70]:
items = pd.read_parquet("data/seven_names.parquet.gzip")["item_no"].values.tolist()
np.random.shuffle(items)
#print(items)
inp = f"dog and cat are similar, why? Which of the following is the correct answer? animal describes dog and cat, clothing describes dog and cat, vehicle describes dog and cat?"
tar = 'animal describes dog and cat'
inp1 = inp  + ', '.join(items)

inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)
print("Input: " + inp)
print("Target: " + tar)
print("Output: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Input: dog and cat are not similar, why? Which of the following is the correct answer? animal describes dog and cat, mammal describes dog and cat, canine describes dog and cat?
Target: animal describes dog and cat
Output: dog and cat are not similar


In [67]:
items = pd.read_parquet("data/seven_names.parquet.gzip")["item_no"].values.tolist()
np.random.shuffle(items)
#print(items)
inp = f"description: 'red lantern'\nfor the description above, what item_no is the correct match? 90494950, 19456165?"
tar = '90494950'
inp1 = inp  + ', '.join(items)

inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)
print("Input: " + inp)
print("Target: " + tar)
print("Output: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Input: description: 'red lantern'
for the description above, what item_no is the correct match? 90494950, 19456165?
Target: 90494950
Output: 90494950, 90494950, 90494950


In [ ]:
embeddings = list(model.named_parameters())[0][1]
lm_head = list(model.named_parameters())[-1][1]
n_new = 100
og = embeddings[:-n_new,:].detach().cpu()
new = embeddings[-n_new:,:].detach().cpu()
og_vals = np.zeros((2048, 30))
og_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(og[:,i], 30, normed=True)
    og_vals[i] = val
    og_bins[i] = (bin[1:]+bin[:-1])/2
new_vals = np.zeros((2048, 30))
new_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(new[:,i], 30, normed=True)
    new_vals[i] = val
    new_bins[i] = (bin[1:]+bin[:-1])/2
for i in range(2048):
    plt.plot(og_bins[i], og_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.title("Embedding Layer")
plt.show()
for i in range(2048):
    plt.plot(new_bins[i], new_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.title("New EM")
plt.show()
n_new = 100
og = lm_head[:-n_new,:].detach().cpu()
new = lm_head[-n_new:,:].detach().cpu()
og_vals = np.zeros((2048, 30))
og_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(og[:,i], 30, normed=True)
    og_vals[i] = val
    og_bins[i] = (bin[1:]+bin[:-1])/2
new_vals = np.zeros((2048, 30))
new_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(new[:,i], 30, normed=True)
    new_vals[i] = val
    new_bins[i] = (bin[1:]+bin[:-1])/2
for i in range(2048):
    plt.plot(og_bins[i], og_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
#plt.ylim(0, 15)
plt.title("Language Model Head")
plt.show()
for i in range(2048):
    plt.plot(new_bins[i], new_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.ylim(0, 10)
plt.title("New LM")
plt.show()

In [ ]:
df = pd.read_parquet('data/miniprompts005.parquet.gzip')
items = pd.read_parquet("data/item_no_100.parquet.gzip")["item_no"].values.tolist()
w = 1123
print(df['input'].iloc[w][:-12] + ", " + ', '.join(items))
print(df['target'].iloc[w])
print(items.remove(items[0]))


In [6]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    BeamSearchScorer,
)
# instantiate logits processors
logits_processor = LogitsProcessorList(
    [
        MinLengthLogitsProcessor(100, eos_token_id=model.config.eos_token_id),
    ]
)

In [57]:
w = 1123
items = pd.read_parquet("data/item_no_100.parquet.gzip")["item_no"].values.tolist()
#items.remove('30493350')
#inp = "query: 'artificial potted plant'. for the above query what item_no is the best match?"
inp = "What are top 3 descriptions of a whale, most relevant first ? a. animal , mammal ,lives in ocean b. mammal, animal , lives in ocean , c. lives in ocean , mammal , animal"
inp1 = inp  + ', '.join(items)
#inp = "99429408 and 50401335 are dining tables. is the previous sentence correct? yes or no?"
inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)#, logits_processor=logits_processor)
print("Q: " + inp)
print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))
#print("A: " + ', '.join((dict.fromkeys((tokenizer.decode(outputs[0], skip_special_tokens=True).split(','))))))

Q: What are top 3 descriptions of a whale, most relevant first ? a. animal , mammal ,lives in ocean b. mammal, animal , lives in ocean , c. lives in ocean , mammal , animal
A: b


In [ ]:
names = pd.read_parquet("data/item_names_100.parquet.gzip").values.tolist()
for i, item in enumerate(names):
    inp = f"describe what you know about {item[0]}"
    inputs = tokenizer.encode(
        inp, return_tensors='pt')
    inputs = inputs.to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(inputs)
    print(item[1])
    print("Q: " + inp)
    print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("")
    print("")
    print("**************************************************************")
    print("")

In [14]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
cos2 = torch.nn.functional.cosine_similarity

In [37]:
v.shape

torch.Size([32100, 2048])

In [44]:
r = cos2(v, lm[-93], dim=-1)
[tokenizer.decode(torch.tensor(r).topk(10, largest=True)[1], skip_special_tokens=False)]

/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


['duplicate Yes Brown yes Pra Sage herbal woodland 2.233']

In [52]:
items = pd.read_parquet("data/seven_names.parquet.gzip")["item_no"].values.tolist()
l = ["furniture", "couch", "sofa", "chair", "shelf", "kitchen", "table", "bed"]
inputs = tokenizer.batch_encode_plus(
    l, return_tensors="pt", padding=True
).to('cuda:0')
inds = torch.tensor([inputs['input_ids'][i].max() for i in range(len(l))])
cos = torch.nn.functional.cosine_similarity
emb = list(model.named_parameters())[0][1]
lm = list(model.named_parameters())[-1][1]
emb_old_tokens = emb[:-101]
lm_old_tokens = lm[:-101]

#tt = emb[inds]
#tt = lm[inds]

emb_new_tokens = emb[-101:]
lm_new_tokens = lm[-101:]

emb_top10 = []
lm_top10 = []
for i in range(len(emb_new_tokens)):
    new_emb_token = emb_new_tokens[i]
    new_lm_token = lm_new_tokens[i]

    emb_top10_similar = torch.tensor(cos(new_emb_token, emb_old_tokens, dim=-1)).topk(10, largest=True)
    lm_top10_similar = torch.tensor(cos(new_lm_token, lm_old_tokens, dim=-1)).topk(10, largest=True)

    emb_top10 += [[tokenizer.decode(emb_top10_similar[1], skip_special_tokens=False)]]
    lm_top10 += [[tokenizer.decode(lm_top10_similar[1], skip_special_tokens=False)]]
print(emb_top10)
print(lm_top10)

/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[['hausen împotrivensemble Mueller Bremen corpulèle cone Verpackung proposer'], ['intégration proximité weit proche Reparatur familiei réussi Wohnung intimacykleinen'], ['semnal Leipzig Frisch logistic intéressé primi wichtiger Strategimodalité bumbac'], ['humorousbodenträglichépouse plein invisibleAgenceanbieter Beziehung protecți'], ['trecIntensivPartidulmpf comisikultur relay succesneuveturism'], ['wächst médicament häufigdessus pantofi Freundhänge deutlich keiner Mängel'], ['Gear Choir touristique Einfluss Jahres Werk Leuteentraîne seara hypothesis'], ['Gruppeässe authorizationASAaţii squad activities Kooperation participatedşte'], ['Plumber répondre rejoindre Rücken bilan Telefonnummer spezialisiertpflanzen Strategiinscrire'], ['gelassenzin BibliothekAuswirkungen Spaß cérémonie geöffnet Distributionimprimer recognized'], ['schätze fizic nausea inexpensive simptome genausoinformatiquesigurăești horrific'], ['véritéblieben frumoasa expérience situations Panorama Healing Marathon mil

In [53]:
queries = pd.read_parquet("data/item_namequeries100.parquet.gzip")
df = pd.DataFrame({'item_no':items,'top_10_emb':emb_top10, 'top_10_lm':lm_top10})
df = df.merge(queries, how='inner', on='item_no')

FileNotFoundError: [Errno 2] No such file or directory: 'data/item_namequeries100.parquet.gzip'

In [10]:
ddf = df[['item_no', 'name', 'top_10_emb', 'top_10_lm','top10 queries']]

In [7]:
top10

NameError: name 'top10' is not defined

In [11]:
d = pd.DataFrame()
c = 0
c1 = 0
c2 = 0
for i in range(300):
    if c == 0:
        d = d.append(ddf.iloc[c1], ignore_index=True)
        c1 += 1
    if c == 1:
        d = d.append(ddf_ptl.iloc[c2], ignore_index=True)
        c2 += 1
    if c == 2:
        d = d.append({'item_no':" ", 'name':" ", 'top_10_emb':" ", 'top_10_lm':" ",'top10 queries':" "}, ignore_index=True)
        c = -1
    c+=1

In [12]:
d.to_excel('old_vs_ptl_2000_epoch.xlsx')

In [ ]:
top3 = []
top32n = 100*[[]]
for i, j in enumerate(top102):
    top3 = [j[0][0:8], j[0][8:16], j[0][16:]]
    top32n[i] = [queries[queries['item_no']==top3[0]]['name'].iloc[0], queries[queries['item_no']==top3[1]]['name'].iloc[0], queries[queries['item_no']==top3[2]]['name'].iloc[0]] 

In [54]:
df = pd.DataFrame({'item_no':items+['item_no'],'top_10_emb':emb_top10, 'top_10_lm':lm_top10})


In [55]:
df

,item_no,top_10_emb,top_10_lm
0,40278675,[hausen împotrivensemble Mueller Bremen corpul...,[pier legs leg colonne Legcorrespondinga feett...
1,40503607,[intégration proximité weit proche Reparatur f...,[rugretaining carpetatexturedrugstufttuftedpre...
2,70461583,[semnal Leipzig Frisch logistic intéressé prim...,[amugcorrespondingforestrydisplayingpreserving...
3,40494491,[humorousbodenträglichépouse plein invisibleAg...,[a externemugenceinte Dog accessory durable ea...
4,50460952,[trecIntensivPartidulmpf comisikultur relay su...,[aabriur shelfdescendingmug Shelf yacht shelve...
5,40527397,[wächst médicament häufigdessus pantofi Freund...,[a throwfuzmittenr Mannschaftmodifying fuzzycl...
6,29303964,[Gear Choir touristique Einfluss Jahres Werk L...,[recettesstoringthèsehüttetigeglückfidelitybmw...
7,19485605,[Gruppeässe authorizationASAaţii squad activit...,[alocatingassemblagecombiningkontoenappareil K...
8,60371762,[Plumber répondre rejoindre Rücken bilan Telef...,[BOOKvases Books Book bookstrich Vintagevenant...
9,39398933,[gelassenzin BibliothekAuswirkungen Spaß cérém...,[télévision HDMI dvd televizor supraveghere ac...


In [13]:
tokenizer.decode(32200)

'item_no'